# Portfolio Optimization Problem
- In our case, we are solving a risk minimization problem using a conic programming approach, where you define the objective function as a risk measure and enforce constraints based on asset weights and portfolio performance.
- Conic optimization allows the efficient solution of problems where the objective or constraints can be expressed in terms of convex cones. The power cone constraints used are a natural way to handle the non-linearity introduced by the Kaniadakis logarithm and other variables.
- Kaniadakis logarithmic function introduces a degree of non-linearity that helps capture non-standard risk measures. The use of ln_k allow us to model risk in a more flexible way than traditional measures like variance or Value at Risk

In [1]:
import numpy as np
import cvxpy as cv
import yfinance as yf
import pandas as pd
import warnings

In [2]:
warnings.filterwarnings("ignore")
# Sets pandas display format for floating-point numbers to percenta
pd.options.display.float_format = '{:.4}%'.format

#start and end date of ananlysis
start = "2016-01-01"
end = "2019-12-31"

# List of asset tickers to analyze
assets = ['TGT', 'CMCSA', 'CPB', 'MO', 'T', 'BAX', 'BMY',
          'MSFT', 'SEE', 'VZ', 'CNP', 'NI', 'GE', 'GOOG']
assets.sort()

# Download historical adjusted close prices for the assets
data = yf.download(assets, start = start, end = end)
data = data.loc[:,('Adj Close', slice(None))]
# Set column names to asset tickers
data.columns = assets

# Calculate daily returns from adjusted close prices
# Compute percentage change and drop missing values
Y = data[assets].pct_change().dropna()
display(Y.head())

[*********************100%***********************]  14 of 14 completed


,BAX,BMY,CMCSA,CNP,CPB,GE,GOOG,MO,MSFT,NI,SEE,T,TGT,VZ
Date,,,,,,,,,,,,,,
2016-01-05 00:00:00+00:00,0.004035%,0.01969%,0.0001799%,0.009305%,0.003678%,0.0009768%,0.0009976%,0.02021%,0.004562%,0.01588%,0.009758%,0.006987%,0.01754%,0.01373%
2016-01-06 00:00:00+00:00,0.002412%,-0.01756%,-0.007727%,-0.01247%,-0.001736%,-0.01594%,0.0014%,0.01059%,-0.01817%,0.005547%,-0.01565%,0.003108%,-0.01016%,-0.009035%
2016-01-07 00:00:00+00:00,-0.01657%,-0.0277%,-0.01105%,-0.01977%,-0.01221%,-0.04231%,-0.02317%,-0.01741%,-0.03478%,-0.02207%,-0.03156%,-0.01615%,-0.0027%,-0.005492%
2016-01-08 00:00:00+00:00,-0.01604%,-0.02542%,0.001099%,-0.002241%,0.005706%,-0.01795%,-0.01641%,0.00172%,0.003067%,-0.001539%,-0.001448%,0.0008951%,-0.03384%,-0.009719%
2016-01-11 00:00:00+00:00,-0.01685%,-0.01022%,0.0009146%,-0.01179%,0.005674%,0.004569%,0.002183%,0.02095%,-0.0005735%,0.01644%,-0.00145%,0.01222%,0.01457%,0.0058%


In [3]:
# Function to calculates the Kaniadakis logarithm
# Define the Kaniadakis logarithm function
def ln_k(x, kappa=0.3):
    return (x**kappa - x**(-kappa))/(2*kappa)

In [4]:
# Convert returns DataFrame to a NumPy array for optimization
returns = Y.to_numpy()
# Get the number of observations (T) and assets (n)
T, n = Y.shape


In [5]:
# Defining initial variables
w = cv.Variable((n, 1)) # Portfolio weights (n assets)
X = returns @ w # Portfolio returns as a product of returns and weights
t = cv.Variable((1, 1)) # Scalar risk component
z = cv.Variable((1, 1), nonneg=True) # Non-negative scalar risk component
alpha = 0.05 # A parameter for the Kaniadakis logarithm
kappa = 0.3 # A parameter for the Kaniadakis logarithm
# Auxiliary variable for conic constraint
omega = cv.Variable((T, 1))
psi = cv.Variable((T, 1))
theta = cv.Variable((T, 1))
epsilon = cv.Variable((T, 1))
# Vector of ones with length T
onesvec = np.ones((T, 1))


Constraints:
 - constraints are defined using 3D power cones, which are a generalization of standard second-order cones. These cones are useful in modeling certain types of convex constraints that involve powers, and they ensure that the variables remain within a certain convex set.
 - In these constraints model the relationship between the variables in the generalized risk measure and enforce that the portfolio returns (X),
 - The auxiliary variables (omega, theta, epsilon, and psi) should remain in a valid range. 
 - The parameter kappa is used here to introduce a degree of non-linearity in the problem.
 - Linear inequality constraint enforces that the portfolio's total risk (represented by X, t, epsilon, and omega) does not exceed a certain threshold. The coefficients of 1000 are used for numerical stability, ensuring that the solver handles the scaling appropriately.
 - Sum of weights constraint ensures that the sum of all asset weights (w) in the portfolio is equal to 1, which is a standard constraint in portfolio optimization, as the weights represent the proportion of the total capital invested in each asset.
 - Non-negativity constraint on weights ensures that the portfolio weights are non-negative, implying that short selling is not allowed. In other words, you can only invest positively in each asset, without borrowing.

In [6]:
# Defining constraints for the optimization problem
constraints = [
    # Power cone constraints for non-linear convex relations
    cv.PowCone3D(
        z * (1 + kappa) / (2 * kappa) * onesvec, # Conic variable related to z
        psi * (1 + kappa) / kappa,  # Conic variable related to psi
        epsilon,  # Conic variable related to epsilon
        1 / (1 + kappa), # Exponent for the power cone
    ),
    cv.PowCone3D(
        omega / (1 - kappa), # Conic variable related to omega
        theta / kappa, # Conic variable related to theta
        -z / (2 * kappa) * onesvec, # Conic variable related to -z
        (1 - kappa), # Exponent for the power cone
    ),
      # Linear inequality constraint to ensure risk does not exceed a threshold
    -X * 1000 - t * 1000 + epsilon * 1000 + omega * 1000 <= 0,
    # Sum of portfolio weights must be equal to 1
    cv.sum(w) == 1,
    # Portfolio weights must be non-negative
    w >= 0
]

Objective function
- t is a scalar variable that represents the "risk-free" component or a baseline risk value.
- z * ln_k(1 / (alpha * T), kappa) represents a risk measure based on the Kaniadakis logarithm. The Kaniadakis logarithm, denoted by ln_k, is a generalized logarithmic function that captures non-extensive statistical properties, and here it is used to model a custom risk measure.
- cv.sum(psi + theta): These are auxiliary variables contributing to the total risk and capturing deviations in the portfolio returns.

Cost function:
- The cost function for your optimization problem is the quantity that you aim to minimize.
- The cost function represents a risk measure that combines various components to capture the total risk associated with the portfolio.

In [7]:
# Total risk to be minimized
risk = t + z * ln_k(1 / (alpha * T), kappa) + cv.sum(psi + theta)
# Setting the objective to minimize risk
objective = cv.Minimize(risk)

In [8]:
# Solving problem
prob = cv.Problem(objective, constraints)
prob.solve()

0.024138151746282213

In [10]:
# Extract and display the optimal portfolio weights
weights = pd.DataFrame(w.value, index=assets, columns=['Weights'])
display(weights)

,Weights
BAX,2.266e-08%
BMY,0.09984%
CMCSA,0.06928%
CNP,0.1209%
CPB,0.1522%
GE,4.458e-09%
GOOG,0.05454%
MO,9.735e-09%
MSFT,2.958e-09%
NI,0.1721%


Summary
- This is a problem of optimization where we have to optimize the solution but also obey the certain constrains, making this a Constrained optimization problem
- Power cone is used to hadle non-linear convex relationship in the way of risk measures, making this a Conic Programming problem
- The Kaniadakis logarithm introduces non-linearities that are convex in nature. Conic programming can efficiently handle such non-linear convex functions through the power cone constraints
- Constraints in your problem are the sum of weights constraint, non-negativity constraint, and the conic constraints (power cones), are all convex. This ensures that the overall problem remains convex.